# 🔥 How to use SOTA Focoos Computer Vision Models

## 🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

# 🎨 There are three ways to use a model:


1. Use the model directly in PyTorch for full flexibility using `ModelManager`.
2. Deploy the model in production with an optimized export for your preferred inference runtime.
3. Effortlessly run your model on Focoos cloud using the `FocoosHUB` and `RemoteModel` class.

## 🔥 Torch Inference

This section demonstrates how to perform local inference using a plain Pytorch model.
We will load a model and then run inference on a sample image.

First, let's get a model. We need to use the `ModelManager` that will take care of instaciating the right model starting from a pre-trained models, a model ref or a folder 

In [ ]:
from focoos import ModelManager

model_ref = "fai-detr-l-obj365"  # use any of your models here

model = ModelManager.get(model_ref)

You can now run the model by simply passing it an image

In [ ]:
from pprint import pprint

from PIL import Image

im_path = "https://public.focoos.ai/samples/pexels-abby-chung.jpg"

detections = model.infer(im_path, annotate=True)  # annotate=True will return an annotated image with the detections
pprint(detections)
Image.fromarray(detections.image)

How fast is this model locally? We can compute it's speed by using the benchmark utility.

In [ ]:
model.benchmark(iterations=10, size=640)

## 🔨 Optimized Inference

As you can see, using the torch model is great, but we can achieve better performance by exporting and running it with a optimized runtime, such as Torchscript, TensorRT, CoreML or the ones available on ONNXRuntime.

In the following cells, we will export the previous model for one of these and run it.

### Torchscript

We already provide multiple inference runtime, that you can see on the `RuntimeTypes` enum. Let's select Torchscript as an example.

It's time to export the model. We can use the export method of the models.

In [ ]:
from focoos import ModelManager, RuntimeType

model_ref = "fai-detr-l-obj365"  # use any of your models here

model = ModelManager.get(model_ref)
runtime = RuntimeType.TORCHSCRIPT_32  # use any of the supported runtimes
optimized_model = model.export(runtime_type=runtime, image_size=640, overwrite=True)

Let's visualize the output. As you will see, there are not differences from the model in pure torch.

In [ ]:
from PIL import Image

im_path = "https://public.focoos.ai/samples/pexels-abby-chung.jpg"
detections = optimized_model.infer(im_path, annotate=True)
Image.fromarray(detections.image)

But, let's see its latency with a benchmark! 

In [ ]:
optimized_model.benchmark(iterations=10)

Wow! That's a lot faster! And without losing a bit in performance!

You can also try different runtimes. Please note that you need to install the relative packages for onnx and tensorRT for using them.

### ONNX with CUDA

In [ ]:
from focoos import RuntimeType

runtime = RuntimeType.ONNX_CUDA32
optimized_model = model.export(runtime_type=runtime)
im_path = "https://public.focoos.ai/samples/pexels-abby-chung.jpg"
detections = optimized_model.infer(im_path, annotate=True)
Image.fromarray(detections.image)

### 🚀 ONNX with TensorRT

In [ ]:
from focoos import RuntimeType

runtime = RuntimeType.ONNX_TRT16
optimized_model = model.export(runtime_type=runtime)

detections = optimized_model.infer(im_path, annotate=True)
Image.fromarray(detections.image)

# ☁ FocoosHUB and Remote Inference
To use the remote inference feature, you need a Focoos API key.
Sign up at https://app.focoos.ai/ and find your personal API key on your profile page.


In [ ]:
import os

from focoos import FocoosHUB

FOCOOS_API_KEY = os.getenv(
    "FOCOOS_API_KEY"
)  # write here your API key, otherwise it will be loaded automatically from env vars
hub = FocoosHUB(api_key=FOCOOS_API_KEY)

## 📦 See your models on HUB
You can see the models available for you on the platform with an intuitive user interface.
However, you can also list them using the Hub functionalities.


In [ ]:
from pprint import pprint

pprint(hub.list_remote_models())

## 🌍 Remote Inference

In this section, you'll run a model on the Focoos' servers instead of on your machine. The image will be packed and sent on the network to the servers, where it is processed and the results is retured to your machine, all in few milliseconds. 

In [ ]:
model_ref = "fai-detr-l-obj365"  # use any of your models here

remote_model = hub.get_remote_model(model_ref)

In [ ]:
from PIL import Image

im_path = "https://public.focoos.ai/samples/pexels-abby-chung.jpg"

detections = remote_model.infer(
    im_path, annotate=True
)  # annotate=True will return an annotated image with the detections

pprint(detections)

Image.fromarray(detections.image)